In [ ]:
from PIL import Image, UnidentifiedImageError
import numpy as np
from skimage.measure import shannon_entropy

def process_image(input_path, block_size, output_path):
    try:
        # Open the image
        image = Image.open(input_path)
    except UnidentifiedImageError:
        print(f"Error: Cannot identify image file '{input_path}'.")
        return
    except Exception as e:
        print(f"An error occurred while opening the image: {e}")
        return

    try:
        # Get the width and height of the image
        width, height = image.size

        # Create a new white canvas of the same size as the input image
        white_canvas = Image.new('RGB', (width, height), 'white')

        # Process the image blocks
        for y in range(0, height, block_size):
            for x in range(0, width, block_size):
                # Create the block
                box = (x, y, min(x + block_size, width), min(y + block_size, height))
                block = image.crop(box)

                # Convert the block to a grayscale NumPy array for entropy calculation
                gray_block = np.array(block.convert('L'))

                # Calculate the entropy of the block
                entropy = shannon_entropy(gray_block)

                # If entropy is higher than 3.0, copy the block to the white canvas
                if entropy > 3.0:
                    white_canvas.paste(block, box)

        # Save the white canvas image
        white_canvas.save(output_path)
        print(f"Output image saved to '{output_path}'.")

    except Exception as e:
        print(f"An error occurred during processing: {e}")

# Example usage
input_path = 'test/Te-gl_0010.jpg'
block_size = 16  # You can adjust the block size
output_path = 'output_image.jpg'
process_image(input_path, block_size, output_path)


In [1]:
import numpy as np
from qiskit_aer.noise import pauli_error, NoiseModel, reset_error
from qiskit import QuantumCircuit, transpile
from qiskit import *
from qiskit_aer import Aer

# Step 4: Create a quantum circuit
qc = QuantumCircuit(17, 8)  # 17 qubits and 8 classical bits
for qubit in range(8):
    qc.rx(np.pi/2, qubit)         # Apply Rx(pi/2) gate to each qubit
    qc.ry(3*np.pi/2, qubit)       # Apply Ry(3*pi/2) gate to each qubit
    qc.rz(np.pi, qubit)           # Apply Rz(pi) gate to each qubit

for qubit in range(8, 16):
    qc.u(np.pi/2, 0, np.pi, qubit)

# Step 1: Define the reset probabilities
prob0 = 0.9  # Probability of not resetting to |0⟩

# Step 1: Define the error probabilities
phase_flip_error_rate_rx = 0.5  # Probability of phase flip error for Rx(pi/2)
phase_flip_error_rate_ry = 0.5  # Probability of phase flip error for Ry(3*pi/2)
phase_flip_error_rate_rz = 0.5  # Probability of phase flip error for Rz(pi)

# Step 2: Create phase flip error channels using pauli_error function
phase_flip_error_rx = pauli_error([('Z', phase_flip_error_rate_rx), ('I', 1 - phase_flip_error_rate_rx)])
phase_flip_error_ry = pauli_error([('Z', phase_flip_error_rate_ry), ('I', 1 - phase_flip_error_rate_ry)])
phase_flip_error_rz = pauli_error([('Z', phase_flip_error_rate_rz), ('I', 1 - phase_flip_error_rate_rz)])

reset_channel = reset_error(prob0)

# Step 3: Create a noise model and add the phase flip error channels to it
noise_model = NoiseModel()
for qubit in range(8):
    noise_model.add_quantum_error(phase_flip_error_rx, 'rx', [qubit])  # Apply to each qubit for Rx(pi/2)
    noise_model.add_quantum_error(phase_flip_error_ry, 'ry', [qubit])  # Apply to each qubit for Ry(3*pi/2)
    noise_model.add_quantum_error(phase_flip_error_rz, 'rz', [qubit])  # Apply to each qubit for Rz(pi)

for qubit in range(16):
    noise_model.add_quantum_error(reset_channel, 'reset', [qubit])  # Apply reset error to qubits

# Perform the XOR operations
qc.barrier()
qc.cy(0, 16)
qc.cy(8, 16)
qc.reset(0)
qc.reset(8)

qc.barrier()
qc.cy(1, 0)
qc.cy(9, 0)
qc.reset(1)
qc.reset(9)

qc.barrier()
qc.cy(2, 8)
qc.cy(10, 8)
qc.reset(2)
qc.reset(10)

qc.barrier()
qc.cy(3, 1)
qc.cy(11, 1)
qc.reset(3)
qc.reset(11)

qc.barrier()
qc.cy(4, 9)
qc.cy(12, 9)
qc.reset(4)
qc.reset(12)

qc.barrier()
qc.cy(5, 2)
qc.cy(13, 2)
qc.reset(5)
qc.reset(13)

qc.barrier()
qc.cy(6, 10)
qc.cy(14, 10)
qc.reset(6)
qc.reset(14)

qc.barrier()
qc.cy(7, 3)
qc.cy(15, 3)
qc.reset(7)
qc.reset(15)
qc.barrier()

qc.measure([16, 0, 8, 1, 9, 2, 10, 3], [0, 1, 2, 3, 4, 5, 6, 7])

# Compile the quantum circuit to be compatible with the backend
simulator = Aer.get_backend('qasm_simulator')
qc_transpiled = transpile(qc, simulator)

# Run the transpiled quantum circuit on the simulator
shots = 2000
result = simulator.run(qc_transpiled, noise_model=noise_model, shots=shots).result()

# Get and print the counts
counts = result.get_counts()

# Convert binary outcomes to integers and print the list
int_counts = [int(outcome, 2) for outcome in counts.keys()]
print(int_counts)
print("Number of distinct outcomes:", len(int_counts))


[233, 218, 198, 122, 124, 127, 85, 17, 33, 76, 148, 39, 71, 102, 176, 84, 193, 140, 158, 43, 194, 92, 132, 126, 173, 53, 134, 249, 162, 220, 253, 172, 75, 111, 237, 79, 160, 15, 20, 6, 36, 13, 28, 186, 12, 119, 115, 196, 120, 221, 100, 89, 177, 195, 58, 69, 4, 205, 197, 229, 57, 114, 191, 232, 234, 5, 143, 83, 11, 168, 161, 189, 246, 175, 24, 166, 91, 81, 207, 239, 163, 41, 208, 236, 228, 35, 223, 48, 219, 169, 151, 18, 55, 222, 25, 34, 56, 98, 203, 214, 235, 174, 133, 129, 138, 3, 209, 7, 32, 38, 159, 180, 248, 201, 204, 21, 199, 62, 251, 8, 109, 243, 68, 200, 103, 164, 147, 181, 227, 136, 144, 64, 29, 22, 23, 93, 116, 170, 2, 10, 37, 225, 150, 152, 110, 46, 240, 250, 245, 226, 97, 139, 252, 99, 242, 211, 192, 230, 137, 153, 107, 59, 165, 178, 74, 255, 182, 94, 80, 82, 54, 60, 106, 61, 90, 70, 52, 135, 156, 155, 187, 14, 128, 224, 112, 215, 1, 67, 31, 213, 101, 113, 231, 26, 141, 130, 65, 212, 131, 9, 125, 96, 16, 104, 244, 86, 206, 44, 27, 51, 117, 105, 63, 202, 72, 108, 216, 121, 19

In [2]:
import numpy as np

# Provided list of integers (S-box)
s_box = int_counts

def walsh_transform(vec):
    n = len(vec)
    if n == 1:
        return vec
    else:
        even = walsh_transform(vec[:n // 2])
        odd = walsh_transform(vec[n // 2:])
        return np.concatenate([even + odd, even - odd])

def nonlinearity(s_box):
    n = len(s_box)
    assert n == 256
    max_nonlinearity = 0
    
    for i in range(8):
        truth_table = [(s_box[j] >> i) & 1 for j in range(n)]
        walsh_spectrum = walsh_transform(np.array(truth_table) * 2 - 1)
        max_nonlinearity = max(max_nonlinearity, 128 - np.max(np.abs(walsh_spectrum)) // 2)
    
    return max_nonlinearity

nonlinearity_value = nonlinearity(s_box)
nonlinearity_value


108

In [6]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

# Provided list of integers (Custom S-box)
custom_s_box = [194, 140, 158, 204, 202, 195, 162, 176, 121, 166, 55, 157, 45, 75, 61, 98, 92, 122, 215, 207, 102, 249, 134, 83, 168, 11, 42, 220, 41, 208, 169, 1, 67, 25, 127, 28, 184, 88, 190, 46, 197, 111, 0, 186, 156, 53, 173, 231, 223, 35, 33, 234, 109, 8, 101, 113, 97, 117, 91, 240, 54, 217, 68, 86, 206, 135, 52, 87, 95, 183, 103, 200, 205, 224, 112, 228, 115, 236, 12, 119, 255, 74, 182, 43, 253, 56, 34, 19, 118, 4, 39, 6, 36, 149, 171, 167, 84, 71, 243, 237, 132, 172, 2, 170, 116, 72, 108, 216, 93, 22, 23, 58, 254, 30, 179, 40, 241, 187, 155, 247, 78, 50, 177, 120, 24, 165, 178, 59, 20, 198, 16, 104, 244, 196, 191, 232, 13, 203, 214, 85, 81, 79, 174, 235, 238, 142, 139, 15, 160, 38, 213, 31, 90, 70, 21, 199, 138, 133, 129, 222, 209, 32, 7, 250, 161, 189, 246, 77, 73, 47, 146, 185, 230, 153, 137, 107, 233, 143, 5, 218, 100, 89, 201, 248, 159, 180, 136, 64, 29, 144, 239, 163, 251, 62, 94, 110, 152, 150, 226, 245, 148, 221, 18, 76, 123, 17, 125, 96, 9, 126, 63, 105, 225, 37, 69, 57, 114, 229, 27, 51, 44, 193, 82, 80, 154, 188, 124, 66, 164, 227, 147, 181, 3, 151, 192, 211, 219, 48, 145, 175, 252, 242, 99, 106, 60, 141, 212, 131, 65, 130, 26, 49, 210, 14, 128, 10]

# Generate the inverse S-box
inverse_s_box = [0] * 256
for i in range(256):
    inverse_s_box[custom_s_box[i]] = i

# Custom AES class with custom S-box
class CustomAES:
    def __init__(self, key):
        self.key = key
        self.cipher = AES.new(self.key, AES.MODE_ECB)

    def _sub_bytes(self, state, sbox):
        return bytes([sbox[b] for b in state])

    def encrypt(self, plaintext):
        plaintext = pad(plaintext, AES.block_size)
        blocks = [plaintext[i:i + AES.block_size] for i in range(0, len(plaintext), AES.block_size)]
        ciphertext = b''

        for block in blocks:
            state = self._sub_bytes(block, custom_s_box)
            ciphertext += self.cipher.encrypt(state)

        return ciphertext

    def decrypt(self, ciphertext):
        blocks = [ciphertext[i:i + AES.block_size] for i in range(0, len(ciphertext), AES.block_size)]
        plaintext = b''

        for block in blocks:
            state = self.cipher.decrypt(block)
            plaintext += self._sub_bytes(state, inverse_s_box)

        return unpad(plaintext, AES.block_size)

# Example usage
key = b'This is a key123'  # 16 bytes key
custom_aes = CustomAES(key)

plaintext = b'Hello, World!'
ciphertext = custom_aes.encrypt(plaintext)
decrypted_text = custom_aes.decrypt(ciphertext)

print('Plaintext:', plaintext)
print('Ciphertext:', ciphertext)
print('Decrypted:', decrypted_text)


Plaintext: b'Hello, World!'
Ciphertext: b'\xe5=\x1e\x9d\x82\xeb\x92\x9a\x9f6\xb6\xd0\x902\x14A'
Decrypted: b'Hello, World!'


In [19]:
import numpy as np
from qiskit_aer.noise import pauli_error, NoiseModel, reset_error
from qiskit import QuantumCircuit, transpile
from qiskit import *
from qiskit_aer import Aer
import math
from collections import Counter

def bits_to_bytes(bits):
    # Pad the bit string to ensure its length is a multiple of 8
    while len(bits) % 8 != 0:
        bits += '0'  # Pad with zeros
    byte_array = bytearray(int(bits[i:i+8], 2) for i in range(0, len(bits), 8))
    return bytes(byte_array)

# Calculate the entropy of the bit string
def calculate_entropy(bit_string):
    length = len(bit_string)
    frequency = Counter(bit_string)
    entropy = -sum((count / length) * math.log2(count / length) for count in frequency.values())
    return entropy


num_qubits = 16

# Create a quantum circuit with the specified number of qubits
qc = QuantumCircuit(num_qubits, num_qubits)

# Apply Hadamard gate to each qubit to create superposition
for i in range(num_qubits):
    qc.h(i)

# Apply CNOT gates between every pair of qubits to create full entanglement
for i in range(num_qubits):
    for j in range(i + 1, num_qubits):
        qc.cx(i, j)

for i in range(num_qubits):
    for j in range(i):
        qc.cx(i, j)
        
        
# Measure all qubits
qc.measure(range(num_qubits), range(num_qubits))

# Display the circuit
qc.draw(output='mpl')

# Use the Aer QASM simulator
simulator = Aer.get_backend('qasm_simulator')

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc, simulator)

# Execute the circuit on the QASM simulator
shots = 16
result = simulator.run(qc_transpiled, shots=shots).result()

# Get and print the counts
counts = result.get_counts()
concatenated_bits = ''.join(counts.keys())
if len(concatenated_bits) < 256:
    concatenated_bits = concatenated_bits.ljust(256, '0')  # Pad with zeros
elif len(concatenated_bits) > 256:
    concatenated_bits = concatenated_bits[:256]  # Truncate to 256 bits

print(concatenated_bits)
print(len(concatenated_bits))
# Convert to bytes
byte_data = bits_to_bytes(concatenated_bits)
print("Byte representation:", byte_data)

# Calculate entropy
entropy_value = calculate_entropy(concatenated_bits)
print("Entropy:", entropy_value)

1100001101111111110001110100011010000010101001000110110111101000000000010111010110011001101000001110000000011111011000111100000101100100100111101101001011000110001100010110100101011010110010110101100000010010000110001001110110110111101011101100100111001000
256
Byte representation: b'\xc3\x7f\xc7F\x82\xa4m\xe8\x01u\x99\xa0\xe0\x1fc\xc1d\x9e\xd2\xc61iZ\xcbX\x12\x18\x9d\xb7\xae\xc9\xc8'
Entropy: 0.9984144269374469
